<a href="https://colab.research.google.com/github/Ikhadija-5/LAB1-AMMI-NLP/blob/main/Khadija_Iddrisu_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [2]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [3]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [4]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [84]:
def softmax(x):
    ### FILL CODE
    return np.exp(x - np.max(x)) / (np.sum(np.exp(x - np.max(x))))

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [159]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    learning_rate = 0.1

    for iter in range(niter):
      ### FILL CODE
      y=np.identity(nlabels)[data[iter][0]]
      y = y.reshape(-1,1)
      x= data[iter][1]
      x= x.reshape(1,-1)
      h = x @ w.T
      y_pred = softmax(h)
      # loss = -np.sum(y@np.log(y_pred).reshape(1,-1))
      loss = y @ (-np.log(y_pred).reshape(1,-1))
      gradient = x * (y_pred.reshape(-1,1) - y)
      w = w - learning_rate * gradient
    
    return w




The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [160]:
def predict(w, x):
    ## FILL CODE
    pred = softmax(np.dot(w,x)).argmax()
    return pred

Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [161]:
def compute_accuracy(w, valid_data):
    ## FILL CODE
    accuracy = 0.0
    for label, x in valid_data:
        ## FILL CODE
        if predict(w,x) == label:
            accuracy+=1
            
    return accuracy/len(valid_data)

In [166]:
print("")
print("** Logistic Regression **")
print("")

word_dict, label_dict = build_dict("train1.txt")
train_data = load_data("train1.txt", word_dict, label_dict)
valid_data = load_data("valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 2500)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **


Validation accuracy: 0.859

